In [5]:
# -*- coding: utf-8 -*-
# MOSFET Stack-Level Material Selector (Metal–Insulator–Semiconductor)
# - Expanded property sets per layer
# - Priority-aware weights, target shifting, overlap bonus, diversity
# - Dielectric per-material scaling & tiny deterministic offsets
# - Hard rejections (sigma, k, Ebd, k_th, Eg, Tm, alpha bounds)
# - Elastic Target Box (ensure ≥1 candidate in-box, minimal symmetric relax)
# - Gradio single-file app

import gradio as gr
import html, math, hashlib

# =============================
# Material libraries (nominal, room-temperature)
# =============================
METALS = [
    {"name":"Al",   "phi":4.28, "sigma":3.5e7,  "alpha":0.0040, "melt":660,  "kth":237},
    {"name":"Cu",   "phi":4.65, "sigma":5.8e7,  "alpha":0.0039, "melt":1085, "kth":401},
    {"name":"W",    "phi":4.55, "sigma":1.8e7,  "alpha":0.0045, "melt":3422, "kth":174},
    {"name":"Mo",   "phi":4.60, "sigma":1.9e7,  "alpha":0.0045, "melt":2623, "kth":138},
    {"name":"Ti",   "phi":4.33, "sigma":2.4e6,  "alpha":0.0038, "melt":1668, "kth":22},
    {"name":"Ta",   "phi":4.25, "sigma":7.4e6,  "alpha":0.0037, "melt":3017, "kth":57},
    {"name":"Ni",   "phi":5.15, "sigma":1.4e7,  "alpha":0.0060, "melt":1455, "kth":91},
    {"name":"Pt",   "phi":5.65, "sigma":9.4e6,  "alpha":0.0039, "melt":1768, "kth":72},
    {"name":"Pd",   "phi":5.60, "sigma":9.5e6,  "alpha":0.0038, "melt":1555, "kth":72},
    {"name":"Ru",   "phi":4.71, "sigma":1.4e7,  "alpha":0.0040, "melt":2334, "kth":117},
    {"name":"Ir",   "phi":5.27, "sigma":2.0e7,  "alpha":0.0038, "melt":2446, "kth":150},
    {"name":"Co",   "phi":5.00, "sigma":1.7e7,  "alpha":0.0043, "melt":1495, "kth":100},
    {"name":"Cr",   "phi":4.50, "sigma":7.9e6,  "alpha":0.0049, "melt":1907, "kth":94},
    {"name":"V",    "phi":4.30, "sigma":5.0e6,  "alpha":0.0043, "melt":1910, "kth":31},
    {"name":"Zr",   "phi":4.05, "sigma":2.3e6,  "alpha":0.0035, "melt":1855, "kth":23},
    {"name":"Hf",   "phi":4.10, "sigma":3.3e6,  "alpha":0.0035, "melt":2233, "kth":23},
    {"name":"TiN",  "phi":4.60, "sigma":4.0e5,  "alpha":0.0008, "melt":2950, "kth":30},
    {"name":"TaN",  "phi":4.70, "sigma":2.5e5,  "alpha":0.0008, "melt":3090, "kth":16},
    {"name":"WN",   "phi":4.80, "sigma":3.0e5,  "alpha":0.0009, "melt":2600, "kth":40},
    {"name":"MoN",  "phi":4.70, "sigma":2.0e5,  "alpha":0.0009, "melt":2200, "kth":32},
    {"name":"TiAlN","phi":4.75, "sigma":3.5e5,  "alpha":0.0008, "melt":3000, "kth":25},
    {"name":"TaC",  "phi":5.00, "sigma":1.0e6,  "alpha":0.0010, "melt":3880, "kth":21},
    {"name":"TiC",  "phi":4.90, "sigma":3.0e6,  "alpha":0.0010, "melt":3160, "kth":21},
    {"name":"TiW",  "phi":4.60, "sigma":6.0e6,  "alpha":0.0035, "melt":2600, "kth":120},
    {"name":"AlCu", "phi":4.50, "sigma":4.5e7,  "alpha":0.0039, "melt":640,  "kth":300},
    {"name":"Au",   "phi":5.10, "sigma":4.1e7,  "alpha":0.0034, "melt":1064, "kth":318},
    {"name":"Ag",   "phi":4.26, "sigma":6.3e7,  "alpha":0.0038, "melt":962,  "kth":429},
    {"name":"Re",   "phi":4.97, "sigma":1.9e7,  "alpha":0.0032, "melt":3186, "kth":48},
    {"name":"Nb",   "phi":4.30, "sigma":6.7e6,  "alpha":0.0037, "melt":2477, "kth":54},
    {"name":"Os",   "phi":5.10, "sigma":1.1e7,  "alpha":0.0035, "melt":3033, "kth":88},
]

# Dielectrics with kth (W/m·K), dEc/dEv (band offsets to Si, eV)
DIELECTRICS = [
    {"name":"SiO2",   "k":3.90,  "Eg":8.90, "Ebd":1.00, "k_texp":-0.10, "Ebd_slope":-0.0010, "kth":1.4, "dEc":3.1, "dEv":4.7},
    {"name":"Si3N4",  "k":7.50,  "Eg":5.00, "Ebd":0.80, "k_texp":-0.10, "Ebd_slope":-0.0011, "kth":20,  "dEc":2.1, "dEv":2.8},
    {"name":"Al2O3",  "k":9.00,  "Eg":8.80, "Ebd":0.80, "k_texp":-0.10, "Ebd_slope":-0.0010, "kth":30,  "dEc":2.8, "dEv":2.0},
    {"name":"HfO2",   "k":22.00, "Eg":5.80, "Ebd":1.20, "k_texp":-0.09, "Ebd_slope":-0.0010, "kth":1.0, "dEc":1.5, "dEv":3.0},
    {"name":"ZrO2",   "k":25.00, "Eg":5.80, "Ebd":1.00, "k_texp":-0.09, "Ebd_slope":-0.0010, "kth":2.0, "dEc":1.4, "dEv":2.7},
    {"name":"La2O3",  "k":27.00, "Eg":4.30, "Ebd":0.80, "k_texp":-0.12, "Ebd_slope":-0.0013, "kth":2.0, "dEc":1.0, "dEv":2.0},
    {"name":"TiO2",   "k":60.00, "Eg":3.20, "Ebd":0.50, "k_texp":-0.12, "Ebd_slope":-0.0013, "kth":7.0, "dEc":0.1, "dEv":2.9},
    {"name":"Ta2O5",  "k":25.00, "Eg":4.40, "Ebd":0.60, "k_texp":-0.10, "Ebd_slope":-0.0012, "kth":3.0, "dEc":0.5, "dEv":2.6},
    {"name":"h-BN",   "k":4.00,  "Eg":5.90, "Ebd":1.20, "k_texp":-0.08, "Ebd_slope":-0.0008, "kth":60,  "dEc":1.7, "dEv":2.2},
    {"name":"Y2O3",   "k":13.90, "Eg":5.60, "Ebd":0.87, "k_texp":-0.10, "Ebd_slope":-0.0010, "kth":5.0, "dEc":1.8, "dEv":2.6},
    {"name":"Gd2O3",  "k":14.50, "Eg":5.40, "Ebd":0.89, "k_texp":-0.12, "Ebd_slope":-0.0012, "kth":5.0, "dEc":1.6, "dEv":2.4},
    {"name":"CeO2",   "k":26.00, "Eg":3.20, "Ebd":0.70, "k_texp":-0.10, "Ebd_slope":-0.0012, "kth":12,  "dEc":0.3, "dEv":2.2},
    {"name":"Sc2O3",  "k":15.20, "Eg":6.00, "Ebd":0.88, "k_texp":-0.11, "Ebd_slope":-0.0011, "kth":4.0, "dEc":1.7, "dEv":2.6},
    {"name":"MgO",    "k":9.80,  "Eg":7.80, "Ebd":0.90, "k_texp":-0.10, "Ebd_slope":-0.0010, "kth":60,  "dEc":2.8, "dEv":3.8},
    {"name":"CaF2",   "k":8.40,  "Eg":12.0, "Ebd":0.90, "k_texp":-0.08, "Ebd_slope":-0.0010, "kth":9.0, "dEc":3.5, "dEv":4.5},
    {"name":"LaAlO3", "k":24.00, "Eg":5.60, "Ebd":0.80, "k_texp":-0.10, "Ebd_slope":-0.0011, "kth":12,  "dEc":1.4, "dEv":2.3},
    {"name":"SrTiO3", "k":200.0, "Eg":3.20, "Ebd":0.30, "k_texp":-0.15, "Ebd_slope":-0.0015, "kth":12,  "dEc":-0.1,"dEv":2.5},
    {"name":"BST",    "k":300.0, "Eg":3.20, "Ebd":0.30, "k_texp":-0.16, "Ebd_slope":-0.0016, "kth":3.0, "dEc":0.0, "dEv":2.4},
    {"name":"BaTiO3", "k":1200., "Eg":3.20, "Ebd":0.20, "k_texp":-0.20, "Ebd_slope":-0.0020, "kth":6.0, "dEc":0.0, "dEv":2.4},
    {"name":"SiON",   "k":5.50,  "Eg":7.00, "Ebd":1.00, "k_texp":-0.10, "Ebd_slope":-0.0010, "kth":2.0, "dEc":2.0, "dEv":3.5},
    {"name":"HfSiO",  "k":12.20, "Eg":6.00, "Ebd":1.02, "k_texp":-0.09, "Ebd_slope":-0.0010, "kth":2.0, "dEc":2.0, "dEv":3.0},
    {"name":"ZrSiO",  "k":11.60, "Eg":6.10, "Ebd":0.98, "k_texp":-0.09, "Ebd_slope":-0.0010, "kth":2.0, "dEc":2.1, "dEv":3.1},
    {"name":"HfZrO2", "k":30.00, "Eg":5.50, "Ebd":0.90, "k_texp":-0.10, "Ebd_slope":-0.0011, "kth":1.2, "dEc":1.2, "dEv":2.7},
    {"name":"AlN",    "k":9.00,  "Eg":6.10, "Ebd":0.80, "k_texp":-0.09, "Ebd_slope":-0.0010, "kth":140, "dEc":2.1, "dEv":0.7},
    {"name":"c-BN",   "k":5.00,  "Eg":6.40, "Ebd":1.20, "k_texp":-0.08, "Ebd_slope":-0.0009, "kth":740, "dEc":1.8, "dEv":2.5},
    {"name":"SiC-oxide","k":7.00,"Eg":8.00, "Ebd":0.80, "k_texp":-0.10, "Ebd_slope":-0.0010, "kth":4.0, "dEc":2.5, "dEv":3.5},
    {"name":"ZrO2:Y", "k":30.00, "Eg":5.50, "Ebd":0.90, "k_texp":-0.10, "Ebd_slope":-0.0011, "kth":2.0, "dEc":1.3, "dEv":2.6},
    {"name":"Ta2O5:N","k":30.00, "Eg":4.60, "Ebd":0.70, "k_texp":-0.10, "Ebd_slope":-0.0012, "kth":3.0, "dEc":0.6, "dEv":2.5},
    {"name":"TiO2:Al","k":45.00, "Eg":3.50, "Ebd":0.60, "k_texp":-0.12, "Ebd_slope":-0.0013, "kth":10,  "dEc":0.2, "dEv":2.8},
    {"name":"Al2O3:H","k":9.50,  "Eg":8.80, "Ebd":0.90, "k_texp":-0.10, "Ebd_slope":-0.0010, "kth":35,  "dEc":2.8, "dEv":2.0},
]

SEMICONDUCTORS = [
    {"name":"Si",        "band_gap":1.12, "mobility":1400,  "thermal":148,  "breakdown":0.30, "dim":"3D"},
    {"name":"Ge",        "band_gap":0.66, "mobility":3900,  "thermal":60,   "breakdown":0.10, "dim":"3D"},
    {"name":"GaAs",      "band_gap":1.42, "mobility":8500,  "thermal":46,   "breakdown":0.40, "dim":"3D"},
    {"name":"InP",       "band_gap":1.34, "mobility":5400,  "thermal":68,   "breakdown":0.50, "dim":"3D"},
    {"name":"GaN",       "band_gap":3.40, "mobility":1250,  "thermal":130,  "breakdown":3.30, "dim":"3D"},
    {"name":"4H-SiC",    "band_gap":3.26, "mobility":1000,  "thermal":490,  "breakdown":3.00, "dim":"3D"},
    {"name":"3C-SiC",    "band_gap":2.36, "mobility":800,   "thermal":320,  "breakdown":2.00, "dim":"3D"},
    {"name":"Ga2O3",     "band_gap":4.80, "mobility":150,   "thermal":16,   "breakdown":8.00, "dim":"3D"},
    {"name":"Diamond",   "band_gap":5.50, "mobility":2200,  "thermal":1200, "breakdown":10.0, "dim":"3D"},
    {"name":"SiGe",      "band_gap":1.00, "mobility":2000,  "thermal":100,  "breakdown":0.25, "dim":"3D"},
    {"name":"InGaAs",    "band_gap":0.75, "mobility":10000, "thermal":55,   "breakdown":0.35, "dim":"3D"},
    {"name":"InAs",      "band_gap":0.36, "mobility":30000, "thermal":27,   "breakdown":0.25, "dim":"3D"},
    {"name":"InSb",      "band_gap":0.17, "mobility":77000, "thermal":18,   "breakdown":0.20, "dim":"3D"},
    {"name":"GaSb",      "band_gap":0.73, "mobility":4500,  "thermal":40,   "breakdown":0.30, "dim":"3D"},
    {"name":"GaP",       "band_gap":2.26, "mobility":130,   "thermal":110,  "breakdown":1.20, "dim":"3D"},
    {"name":"AlGaAs",    "band_gap":1.90, "mobility":6000,  "thermal":55,   "breakdown":0.50, "dim":"3D"},
    {"name":"InGaN",     "band_gap":2.00, "mobility":600,   "thermal":150,  "breakdown":2.50, "dim":"3D"},
    {"name":"ZnO",       "band_gap":3.37, "mobility":200,   "thermal":50,   "breakdown":3.00, "dim":"3D"},
    {"name":"ZnSe",      "band_gap":2.70, "mobility":600,   "thermal":19,   "breakdown":1.50, "dim":"3D"},
    {"name":"ZnS",       "band_gap":3.60, "mobility":200,   "thermal":27,   "breakdown":2.00, "dim":"3D"},
    {"name":"CdS",       "band_gap":2.42, "mobility":300,   "thermal":16,   "breakdown":1.00, "dim":"3D"},
    {"name":"CdSe",      "band_gap":1.74, "mobility":700,   "thermal":7,    "breakdown":0.60, "dim":"3D"},
    {"name":"Ga2Se3",    "band_gap":2.02, "mobility":120,   "thermal":7,    "breakdown":0.80, "dim":"3D"},
    {"name":"MoS2",      "band_gap":1.80, "mobility":350,   "thermal":85,   "breakdown":0.70, "dim":"2D"},
    {"name":"WS2",       "band_gap":2.10, "mobility":250,   "thermal":120,  "breakdown":0.80, "dim":"2D"},
    {"name":"WSe2",      "band_gap":1.65, "mobility":250,   "thermal":40,   "breakdown":0.70, "dim":"2D"},
    {"name":"MoSe2",     "band_gap":1.55, "mobility":300,   "thermal":70,   "breakdown":0.70, "dim":"2D"},
    {"name":"MoTe2",     "band_gap":1.10, "mobility":200,   "thermal":35,   "breakdown":0.60, "dim":"2D"},
    {"name":"HfS2",      "band_gap":2.00, "mobility":180,   "thermal":45,   "breakdown":0.80, "dim":"2D"},
    {"name":"In2Se3",    "band_gap":1.30, "mobility":400,   "thermal":10,   "breakdown":0.60, "dim":"2D"},
    {"name":"BP",        "band_gap":0.30, "mobility":1000,  "thermal":34,   "breakdown":0.20, "dim":"2D"},
    {"name":"InSe",      "band_gap":1.25, "mobility":1000,  "thermal":15,   "breakdown":0.30, "dim":"2D"},
]

# =============================
# Scenarios & target windows
# =============================
SCENARIOS = [
    "High speed / RF",
    "Low power logic / portable",
    "Medium/low-voltage analog / sensing",
    "High temperature environment",
    "High voltage / power switching",
    "Ultra-short channel / frontier exploration",
    "General IC (CMOS-friendly)"
]

# Expanded targets: metals add alpha, kth; dielectrics add kth, dEc, dEv
TARGETS_METAL = {
    "High speed / RF":                            {"phi":(4.4,4.9), "sigma":(1e6,6e7), "melt":(600,3000),  "alpha":(5e-4,4.5e-3), "kth":(50,400)},
    "Low power logic / portable":                 {"phi":(4.4,5.0), "sigma":(1e6,6e7), "melt":(600,2000),  "alpha":(5e-4,5.0e-3), "kth":(30,300)},
    "Medium/low-voltage analog / sensing":        {"phi":(4.6,5.2), "sigma":(1e6,5e7), "melt":(800,2500),  "alpha":(5e-4,5.0e-3), "kth":(50,350)},
    "High temperature environment":               {"phi":(4.6,5.3), "sigma":(1e5,5e7), "melt":(1500,3500), "alpha":(5e-4,4.5e-3), "kth":(60,400)},
    "High voltage / power switching":             {"phi":(4.6,5.3), "sigma":(1e6,5e7), "melt":(1500,3500), "alpha":(5e-4,4.5e-3), "kth":(60,400)},
    "Ultra-short channel / frontier exploration": {"phi":(4.5,5.2), "sigma":(1e5,5e7), "melt":(800,3000),  "alpha":(5e-4,4.5e-3), "kth":(40,400)},
    "General IC (CMOS-friendly)":                 {"phi":(4.5,4.9), "sigma":(1e6,6e7), "melt":(800,2000),  "alpha":(5e-4,5.0e-3), "kth":(30,300)},
}
TARGETS_DIEL = {
    "High speed / RF":                            {"k":(8,30),  "Ebd":(0.8,1.3), "Eg":(4.5,9.0), "kth":(1,30), "dEc":(1.0,3.5), "dEv":(1.0,4.5)},
    "Low power logic / portable":                 {"k":(6,20),  "Ebd":(0.8,1.2), "Eg":(5.0,9.0), "kth":(1,20), "dEc":(1.0,3.5), "dEv":(1.0,4.5)},
    "Medium/low-voltage analog / sensing":        {"k":(6,15),  "Ebd":(0.9,1.2), "Eg":(6.0,9.0), "kth":(1,20), "dEc":(1.2,3.5), "dEv":(1.2,4.5)},
    "High temperature environment":               {"k":(8,25),  "Ebd":(0.9,1.3), "Eg":(5.0,9.0), "kth":(2,30), "dEc":(1.0,3.5), "dEv":(1.0,4.5)},
    "High voltage / power switching":             {"k":(15,40), "Ebd":(1.0,1.5), "Eg":(4.5,9.0), "kth":(1,25), "dEc":(1.0,3.5), "dEv":(1.0,4.5)},
    "Ultra-short channel / frontier exploration": {"k":(10,35), "Ebd":(0.8,1.3), "Eg":(4.0,9.0), "kth":(1,25), "dEc":(0.8,3.5), "dEv":(0.8,4.5)},
    "General IC (CMOS-friendly)":                 {"k":(3.5,25),"Ebd":(0.9,1.2), "Eg":(6.0,9.0), "kth":(1,20), "dEc":(1.0,3.5), "dEv":(1.0,4.5)},
}
TARGETS_SEMI = {
    "High speed / RF":                            {"bandgap":(1.0,1.6), "mobility":(3000,10000), "thermal":(80,230),  "breakdown":(0.4,1.5)},
    "Low power logic / portable":                 {"bandgap":(1.0,2.0), "mobility":(500,2000),   "thermal":(80,150),  "breakdown":(0.3,1.0)},
    "Medium/low-voltage analog / sensing":        {"bandgap":(1.0,1.8), "mobility":(800,3000),   "thermal":(60,150),  "breakdown":(0.3,1.0)},
    "High temperature environment":               {"bandgap":(2.0,4.0), "mobility":(300,1500),   "thermal":(100,300), "breakdown":(0.5,2.0)},
    "High voltage / power switching":             {"bandgap":(3.0,5.0), "mobility":(200,1500),   "thermal":(120,500), "breakdown":(2.0,8.0)},
    "Ultra-short channel / frontier exploration": {"bandgap":(1.5,2.2), "mobility":(200,1000),   "thermal":(60,120),  "breakdown":(0.5,1.5)},
    "General IC (CMOS-friendly)":                 {"bandgap":(1.0,1.2), "mobility":(1000,1500),  "thermal":(120,180), "breakdown":(0.3,0.5)},
}

WEIGHTS_METAL = {
    "High speed / RF":                            {"phi":1.0,"sigma":3.0,"melt":1.0,"alpha":1.0,"kth":1.5},
    "Low power logic / portable":                 {"phi":2.5,"sigma":2.0,"melt":1.0,"alpha":1.0,"kth":1.0},
    "Medium/low-voltage analog / sensing":        {"phi":2.5,"sigma":1.5,"melt":1.0,"alpha":1.0,"kth":1.0},
    "High temperature environment":               {"phi":2.0,"sigma":1.0,"melt":3.0,"alpha":1.0,"kth":2.0},
    "High voltage / power switching":             {"phi":2.0,"sigma":1.5,"melt":3.0,"alpha":1.0,"kth":2.0},
    "Ultra-short channel / frontier exploration": {"phi":2.0,"sigma":2.5,"melt":1.5,"alpha":1.0,"kth":1.5},
    "General IC (CMOS-friendly)":                 {"phi":2.5,"sigma":2.0,"melt":1.0,"alpha":1.0,"kth":1.0},
}
WEIGHTS_DIEL = {
    "High speed / RF":                            {"k":3.0,"Ebd":2.0,"Eg":1.5,"kth":1.0,"dEc":1.0,"dEv":1.0},
    "Low power logic / portable":                 {"k":2.0,"Ebd":2.0,"Eg":2.0,"kth":1.0,"dEc":1.0,"dEv":1.0},
    "Medium/low-voltage analog / sensing":        {"k":2.0,"Ebd":2.5,"Eg":2.0,"kth":1.0,"dEc":1.5,"dEv":1.5},
    "High temperature environment":               {"k":2.0,"Ebd":2.5,"Eg":2.0,"kth":1.5,"dEc":1.0,"dEv":1.0},
    "High voltage / power switching":             {"k":3.0,"Ebd":3.0,"Eg":1.5,"kth":1.0,"dEc":1.0,"dEv":1.0},
    "Ultra-short channel / frontier exploration": {"k":3.0,"Ebd":2.0,"Eg":1.5,"kth":1.0,"dEc":1.0,"dEv":1.0},
    "General IC (CMOS-friendly)":                 {"k":2.0,"Ebd":2.0,"Eg":2.0,"kth":1.0,"dEc":1.0,"dEv":1.0},
}
WEIGHTS_SEMI = {
    "High speed / RF":                            {"bandgap":1.0,"mobility":3.0,"thermal":1.0,"breakdown":1.0},
    "Low power logic / portable":                 {"bandgap":3.0,"mobility":2.0,"thermal":1.0,"breakdown":1.0},
    "Medium/low-voltage analog / sensing":        {"bandgap":2.0,"mobility":2.0,"thermal":1.0,"breakdown":1.0},
    "High temperature environment":               {"bandgap":3.0,"mobility":1.0,"thermal":3.0,"breakdown":1.5},
    "High voltage / power switching":             {"bandgap":2.0,"mobility":1.0,"thermal":2.5,"breakdown":3.0},
    "Ultra-short channel / frontier exploration": {"bandgap":3.0,"mobility":2.0,"thermal":1.0,"breakdown":1.0},
    "General IC (CMOS-friendly)":                 {"bandgap":2.0,"mobility":2.0,"thermal":1.0,"breakdown":1.0},
}

SCENARIO_GUIDE_MD = """
**Scenario guide**

- **High speed / RF** — Focus on fast transport/switching (e.g., LNA, PA, mmWave).
- **Low power logic / portable** — Emphasize low leakage/energy (e.g., mobile SoC, wearables).
- **Medium/low-voltage analog / sensing** — Stable and low-noise (e.g., precision analog, biosensors).
- **High temperature environment** — Reliable operation at elevated ambient temperature.
- **High voltage / power switching** — High blocking field and robustness.
- **Ultra-short channel / frontier exploration** — Nanoscale limit/material exploration.
- **General IC (CMOS-friendly)** — Compatibility within the standard CMOS ecosystem.

Note: For each layer, the tool automatically selects **3 key properties** from an **expanded property pool** for the current evaluation.
"""

REASON = {
    "bandgap":"Suppresses intrinsic carriers at high temperature",
    "mobility":"Enables higher drive current and high-frequency performance",
    "thermal":"Removes heat efficiently, improving thermal robustness",
    "breakdown":"Adds electric-field margin and reliability",
    "phi":"Tunes Vth window and suppresses gate leakage",
    "sigma":"Reduces gate resistance and RC delay",
    "melt":"Improves thermal headroom and structural robustness",
    "k":"Boosts Cox and lowers EOT",
    "Ebd":"Raises dielectric field margin and reliability",
    "Eg":"Suppresses tunneling/leakage through the dielectric",
    "alpha":"Lower TCR mitigates resistivity increase at high temperature",
    "kth":"Higher thermal conductivity helps stack heat spreading",
    "dEc":"Larger Si conduction-band offset reduces electron injection/leakage",
    "dEv":"Larger Si valence-band offset reduces hole injection/leakage",
}

# =============================
# Utilities
# =============================
def esc(x): return html.escape(str(x)) if x is not None else "—"

def default_T_range(scenarios, tmax):
    if tmax and tmax > 25:
        return 25.0, float(tmax)
    return (25.0, 200.0) if "High temperature environment" in scenarios else (25.0, 85.0)

def high_field_factor(vdd):
    if not vdd: return 1.0
    if vdd <= 5: return 1.0
    if vdd <= 100: return 0.85
    return 0.7

def mobility_exponent(dim): return 1.5 if dim == "3D" else 1.2
def eg_slope_frac_per_100K(eg300): return 0.05 if eg300 < 2.0 else 0.03

# Stable tiny deterministic per-material offset (process spread)
def _stable_hash_unit(name, prop):
    h = hashlib.md5(f"{name}:{prop}".encode()).hexdigest()
    v = (int(h[:8], 16) / 0xFFFFFFFF) * 2 - 1  # map to [-1, +1]
    return v

def _name_offset(name, prop, scale=0.02):
    return 1.0 + scale * _stable_hash_unit(name, prop)

def diel_room_nominal_with_offset(d, key, scale=0.02):
    return d[key] * _name_offset(d["name"], key, scale=scale)

BASE_TIGHTEN = 0.7
SOFT_DECAY_SCALE = 1.0

def intersect_ranges(rngs):
    lo = max(r[0] for r in rngs)
    hi = min(r[1] for r in rngs)
    return None if hi < lo else (lo, hi)

def shrink_range(rng, factor):
    if rng is None: return None
    lo, hi = rng
    if hi <= lo: return (lo, hi)
    mid = 0.5 * (lo + hi)
    half = 0.5 * (hi - lo) * max(min(factor, 1.0), 0.05)
    return (mid - half, mid + half)

def compute_shrink_factor(scenarios, pr_power, pr_speed, pr_noise, pr_rel, freq, freq_pr, vdd, vdd_pr, tmax, tmax_pr):
    factor = BASE_TIGHTEN
    n = max(len(scenarios), 1)
    factor *= (0.90 ** (n - 1))
    highs = sum([pr_power == "High", pr_speed == "High", pr_noise == "High", pr_rel == "High"])
    factor *= (0.92 ** highs)
    if freq and freq_pr == "High": factor *= 0.95
    if vdd and vdd_pr == "High":   factor *= 0.90
    if tmax and tmax_pr == "High": factor *= 0.90
    return max(0.35, min(factor, 0.9))

def merge_intersect_and_narrow(scenarios, targets_dict, props, shrink_factor):
    out = {}
    for p in props:
        rngs = []
        for sc in scenarios:
            t = targets_dict.get(sc, {})
            if p in t: rngs.append(t[p])
        if not rngs:
            out[p] = None
            continue
        inter = intersect_ranges(rngs)
        if inter is None:
            rngs_sorted = sorted(rngs, key=lambda x: x[1] - x[0])
            out[p] = shrink_range(rngs_sorted[0], shrink_factor)
        else:
            out[p] = shrink_range(inter, shrink_factor)
    return out

def shift_targets(targets, layer, freq, vdd, tmax):
    out = {}
    for p, rng in targets.items():
        if not rng:
            out[p] = None; continue
        lo, hi = rng
        mid = 0.5 * (lo + hi); half = 0.5 * (hi - lo)
        bias = 0.0
        if layer == "semi":
            if p == "mobility" and freq: bias += min(0.25, 0.02 * float(freq))
            if p == "breakdown" and vdd: bias += min(0.35, 0.001 * float(vdd))
            if p == "thermal" and tmax:  bias += min(0.30, 0.002 * float(tmax))
            if p == "bandgap" and vdd:   bias += min(0.10, 0.0003 * float(vdd))
        elif layer == "dielectric":
            if p == "k" and freq:        bias += min(0.20, 0.015 * float(freq))
            if p == "Ebd" and vdd:       bias += min(0.35, 0.001 * float(vdd))
            if p == "Ebd" and tmax:      bias += min(0.20, 0.001 * float(tmax))
            if p == "Eg" and vdd:        bias += min(0.10, 0.0003 * float(vdd))
        else:  # metal
            if p == "sigma" and freq:    bias += min(0.25, 0.02 * float(freq))
            if p == "melt" and tmax:     bias += min(0.30, 0.002 * float(tmax))
        new_mid = mid * (1.0 + bias)
        out[p] = (new_mid - half, new_mid + half)
    return out

def interval_overlap(a, b):
    if (not a) or (not b): return 0.0
    lo = max(a[0], b[0]); hi = min(a[1], b[1])
    return max(0.0, hi - lo)

def score_to_range_soft(value, rng):
    if not rng: return 0.5
    lo, hi = rng
    if hi <= lo: return 1.0
    if lo <= value <= hi: return 1.0
    width = hi - lo
    band = SOFT_DECAY_SCALE * width
    if value < lo: return max(0.0, 1.0 - (lo - value) / max(band, 1e-9))
    else:          return max(0.0, 1.0 - (value - hi) / max(band, 1e-9))

def build_priority_weights(layer, key_props, pr_power, pr_speed, pr_noise, pr_rel,
                           freq, freq_pr, vdd, vdd_pr, tmax, tmax_pr, scenarios):
    w = {p: 1.0 / len(key_props) for p in key_props}
    def bump(p, amt):
        if p in w: w[p] += amt
    sp = {"Low":0.0, "Medium":0.5, "High":1.0}
    if layer == "semi":
        bump("mobility", 0.8 * sp[pr_speed])
        bump("thermal", 0.6 * sp[pr_rel] + 0.6 * sp[tmax_pr])
        bump("breakdown", 0.8 * sp[pr_power] + 0.8 * sp[pr_rel] + 0.8 * sp[vdd_pr])
        bump("bandgap",  0.6 * sp[pr_noise] + 0.4 * sp[pr_rel])
        if freq and freq_pr == "High": bump("mobility", 0.4)
        if vdd  and vdd_pr  == "High": bump("breakdown", 0.4)
        if tmax and tmax_pr == "High": bump("thermal", 0.4)
    elif layer == "dielectric":
        bump("k",   0.7 * sp[pr_speed] + 0.4 * sp[pr_noise])
        bump("Ebd", 0.9 * sp[pr_power] + 0.8 * sp[pr_rel] + 0.9 * sp[vdd_pr])
        bump("Eg",  0.7 * sp[pr_noise] + 0.5 * sp[pr_rel])
        if vdd and vdd_pr == "High": bump("Ebd", 0.4)
        if tmax and tmax_pr == "High": bump("Ebd", 0.2)
    else:  # metal
        bump("sigma", 0.8 * sp[pr_speed])
        bump("melt",  0.8 * sp[pr_rel] + 0.7 * sp[tmax_pr] + 0.6 * sp[pr_power])
        bump("phi",   0.7 * sp[pr_noise])
        bump("sigma", 0.3 * sp[pr_power] + 0.25 * sp[pr_rel] + 0.3 * sp[vdd_pr])
        if "sigma" in w:
            w["sigma"] = max(w["sigma"], 0.25)
    s = sum(w.values())
    return {p: (w[p] / s if s > 0 else 1.0 / len(key_props)) for p in key_props}

def pick_3_keys(layer, scenarios):
    if layer == "metal":
        W = WEIGHTS_METAL; props = ["phi", "sigma", "melt", "alpha", "kth"]
    elif layer == "dielectric":
        W = WEIGHTS_DIEL; props = ["k", "Ebd", "Eg", "kth", "dEc", "dEv"]
    else:
        W = WEIGHTS_SEMI; props = ["bandgap", "mobility", "thermal", "breakdown"]
    agg = {p: 0.0 for p in props}
    for sc in scenarios:
        if sc in W:
            for p, v in W[sc].items():
                if p in agg: agg[p] += v
    top = sorted(agg.items(), key=lambda kv: kv[1], reverse=True)[:3]
    return [p for p, _ in top]

# Environment-based property ranges
def metal_range(m, T_low, T_high):
    def sigma_at(TC): return max(m["sigma"] / (1.0 + m["alpha"] * max(TC - 25.0, 0.0)), 1e3)
    sig_lo, sig_hi = min(sigma_at(T_low), sigma_at(T_high)), max(sigma_at(T_low), sigma_at(T_high))
    out = {"phi": (m["phi"], m["phi"]), "sigma": (sig_lo, sig_hi), "melt": (m["melt"], m["melt"])}
    if "alpha" in m: out["alpha"] = (m["alpha"], m["alpha"])
    if "kth"  in m: out["kth"]  = (m["kth"],  m["kth"])
    return out

def diel_range(d, T_low, T_high):
    T0 = 298.0
    k0   = diel_room_nominal_with_offset(d, "k",  scale=0.02)
    Eg0  = diel_room_nominal_with_offset(d, "Eg", scale=0.02)
    Ebd0 = diel_room_nominal_with_offset(d, "Ebd",scale=0.02)
    k_texp    = d.get("k_texp", -0.10)
    ebd_slope = d.get("Ebd_slope", -0.001)
    def k_at(TK):   return k0  * (TK/T0)**(k_texp)
    def ebd_at(TC): return max(Ebd0 * (1.0 + ebd_slope*max(TC-25.0,0.0)), 0.05)
    TK_low, TK_high = (T_low+273.15), (T_high+273.15)
    k_lo, k_hi     = min(k_at(TK_low), k_at(TK_high)), max(k_at(TK_low), k_at(TK_high))
    ebd_lo, ebd_hi = min(ebd_at(T_low), ebd_at(T_high)), max(ebd_at(T_low), ebd_at(T_high))
    out = {"k": (k_lo, k_hi), "Eg": (Eg0, Eg0), "Ebd": (ebd_lo, ebd_hi)}
    if "kth" in d:  out["kth"] = (d["kth"], d["kth"])
    if "dEc" in d:  out["dEc"] = (d["dEc"], d["dEc"])
    if "dEv" in d:  out["dEv"] = (d["dEv"], d["dEv"])
    return out

def semi_range(s, T_low, T_high, vdd):
    T0 = 298.0
    TK_low, TK_high = (T_low + 273.15), (T_high + 273.15)
    eg0, mu0, k0, ec0 = s["band_gap"], s["mobility"], s["thermal"], s["breakdown"]
    eg_alpha = eg_slope_frac_per_100K(eg0)
    def eg_at(TC): return max(eg0 * (1.0 - max(eg_alpha * ((TC - 25.0) / 100.0), 0.0)), 0.01)
    mexp = mobility_exponent(s["dim"])
    hf = high_field_factor(vdd)
    def mu_at(TK): return mu0 * (TK / T0) ** (-mexp) * hf
    def k_at(TK):  return k0  * (TK / T0) ** (-1.0)
    def ec_at(TC): return max(ec0 * (1.0 - 0.001 * max(TC - 25.0, 0.0)), 0.01)
    eg_lo, eg_hi = min(eg_at(T_low), eg_at(T_high)), max(eg_at(T_low), eg_at(T_high))
    mu_lo, mu_hi = min(mu_at(TK_low), mu_at(TK_high)), max(mu_at(TK_low), mu_at(TK_high))
    k_lo,  k_hi  = min(k_at(TK_low),  k_at(TK_high)),  max(k_at(TK_low),  k_at(TK_high))
    ec_lo, ec_hi = min(ec_at(T_low),  ec_at(T_high)),  max(ec_at(T_low),  ec_at(T_high))
    return {"bandgap": (eg_lo, eg_hi), "mobility": (mu_lo, mu_hi), "thermal": (k_lo, k_hi), "breakdown": (ec_lo, ec_hi)}

# --------- Hard reject rules ----------
def _hard_reject_layer(layer, key_props, ranges, target_ranges):
    def mid(p):
        lo, hi = ranges[p]; return 0.5*(lo+hi)
    if layer == "metal":
        tr_sigma = target_ranges.get("sigma")
        if tr_sigma and "sigma" in key_props and mid("sigma") < 0.5*tr_sigma[0]:
            return True
        tr_melt  = target_ranges.get("melt")
        if tr_melt and "melt" in key_props and mid("melt") < 0.9*tr_melt[0]:
            return True
        tr_kth = target_ranges.get("kth")
        if tr_kth and "kth" in key_props and mid("kth") < 0.5*tr_kth[0]:
            return True
        tr_alpha = target_ranges.get("alpha")
        if tr_alpha and "alpha" in key_props and mid("alpha") > tr_alpha[1]:
            return True
        return False
    if layer == "dielectric":
        tr_k   = target_ranges.get("k")
        tr_ebd = target_ranges.get("Ebd")
        if tr_k   and "k"   in key_props and mid("k")   < 0.6*tr_k[0]:
            return True
        if tr_ebd and "Ebd" in key_props and mid("Ebd") < 0.8*tr_ebd[0]:
            return True
        tr_kth = target_ranges.get("kth")
        if tr_kth and "kth" in key_props and mid("kth") < 0.5*tr_kth[0]:
            return True
        tr_dEc = target_ranges.get("dEc")
        if tr_dEc and "dEc" in key_props and mid("dEc") < tr_dEc[0]:
            return True
        tr_dEv = target_ranges.get("dEv")
        if tr_dEv and "dEv" in key_props and mid("dEv") < tr_dEv[0]:
            return True
        return False
    if layer == "semi":
        tr_th  = target_ranges.get("thermal")
        tr_eg  = target_ranges.get("bandgap")
        if tr_th and "thermal" in key_props and mid("thermal") < 0.5*tr_th[0]:
            return True
        if tr_eg and "bandgap" in key_props and mid("bandgap") < 0.7*tr_eg[0]:
            return True
        return False

# ---------- Elastic-box helpers ----------
def _in_box(mids, targets, props):
    for p in props:
        lo, hi = targets.get(p, (None, None))
        if lo is None:
            continue
        v = mids[p]
        if not (lo <= v <= hi):
            return False
    return True

def _midprops_of(name, ranges_map, props):
    return {p: 0.5*(ranges_map[name][p][0] + ranges_map[name][p][1]) for p in props}

def _needed_relax_ratio(mids, targets, props, eps=1e-9):
    r_need = 0.0
    for p in props:
        lo, hi = targets.get(p, (None, None))
        if lo is None:
            continue
        v = mids[p]
        if v < lo:
            r_need = max(r_need, (lo - v) / max(lo, eps))
        elif v > hi:
            r_need = max(r_need, (v - hi) / max(hi, eps))
    return max(0.0, r_need)

def _relax_targets(targets, r):
    out = {}
    for p, rng in targets.items():
        if not rng:
            out[p] = None; continue
        lo, hi = rng
        out[p] = (lo*(1.0 - r), hi*(1.0 + r))
    return out

def ensure_min_one_in_box(layer, candidates, ranges_map, targets, key_props,
                          base_relax=0.05, max_relax=0.12):
    for c in candidates:
        mids = _midprops_of(c["name"], ranges_map, key_props)
        if _in_box(mids, targets, key_props):
            return targets, c["name"], 0.0
    best = (1e9, None)
    for c in candidates:
        mids = _midprops_of(c["name"], ranges_map, key_props)
        r_need = _needed_relax_ratio(mids, targets, key_props)
        if r_need < best[0]:
            best = (r_need, c["name"])
    r_min, winner = best
    if r_min <= 0:
        return targets, winner, 0.0
    r_use = min(max(r_min, base_relax), max_relax)
    targets_new = _relax_targets(targets, r_use)
    return targets_new, winner, r_use
# -----------------------------------------

def rank_layer(candidates, layer, ranges_by_item, target_ranges, key_props, topk=5, weights=None):
    if weights is None:
        weights = {p: 1.0 / len(key_props) for p in key_props}
    scored = []
    for c in candidates:
        r = ranges_by_item[c["name"]]
        if _hard_reject_layer(layer, key_props, r, target_ranges):
            continue
        score = 0.0
        for p in key_props:
            tr = target_ranges.get(p)
            mid = 0.5 * (r[p][0] + r[p][1])
            prox = score_to_range_soft(mid, tr)
            w = (tr[1] - tr[0]) if tr else 1.0
            ovl = interval_overlap(r[p], tr) / max(w, 1e-9) if tr else 0.0
            score += weights[p] * (0.7 * prox + 0.3 * ovl)
        scored.append((score, c))
    scored.sort(key=lambda x: x[0], reverse=True)

    # Simple diversity bonus
    selected = []
    for sc, c in scored:
        if not selected:
            selected.append((sc, c)); continue
        mids_sel = [
            {p: 0.5 * (ranges_by_item[x[1]["name"]][p][0] + ranges_by_item[x[1]["name"]][p][1]) for p in key_props}
            for x in selected
        ]
        mids_c = {p: 0.5 * (ranges_by_item[c["name"]][p][0] + ranges_by_item[c["name"]][p][1]) for p in key_props}
        sim = max(sum(abs(mids_c[p] - m[p]) for p in key_props) for m in mids_sel)  # larger = more different
        score_div = sc + 0.02 * sim
        selected.append((score_div, c))
        selected = sorted(selected, key=lambda x: x[0], reverse=True)[:topk]
        if len(selected) >= topk: break
    selected.sort(key=lambda x: x[0], reverse=True)
    return [(sc, cc, None) for sc, cc in selected], selected[:topk]

# Formatting helpers
def fmt_range(prop, lo, hi, layer):
    if layer == "metal":
        if prop == "phi":   return f"{lo:.2f}–{hi:.2f} eV"
        if prop == "sigma": return f"{lo:.2e}–{hi:.2e} S/m"
        if prop == "melt":  return f"{int(lo)}–{int(hi)} °C"
        if prop == "alpha": return f"{lo:.4f}–{hi:.4f} 1/°C"
        if prop == "kth":   return f"{lo:.0f}–{hi:.0f} W/m·K"
    elif layer == "dielectric":
        if prop == "k":     return f"{lo:.2f}–{hi:.2f}"
        if prop == "Eg":    return f"{lo:.2f}–{hi:.2f} eV"
        if prop == "Ebd":   return f"{lo:.3f}–{hi:.3f} MV/cm"
        if prop == "kth":   return f"{lo:.0f}–{hi:.0f} W/m·K"
        if prop == "dEc":   return f"{lo:.2f}–{hi:.2f} eV"
        if prop == "dEv":   return f"{lo:.2f}–{hi:.2f} eV"
    else:  # semiconductor
        if prop == "bandgap":  return f"{lo:.2f}–{hi:.2f} eV"
        if prop == "mobility": return f"{int(lo)}–{int(hi)} cm²/V·s"
        if prop == "thermal":  return f"{int(lo)}–{int(hi)} W/m·K"
        if prop == "breakdown":return f"{lo:.2f}–{hi:.2f} MV/cm"
    return f"{lo:.3g}–{hi:.3g}"

def header_with_target(prop, target, layer):
    unit = {"phi":"eV","sigma":"S/m","melt":"°C","k":"","Eg":"eV","Ebd":"MV/cm",
            "bandgap":"eV","mobility":"cm²/V·s","thermal":"W/m·K","breakdown":"MV/cm",
            "alpha":"1/°C","kth":"W/m·K","dEc":"eV","dEv":"eV"}[prop]
    label = {
        "phi":"Work Function (ΦM)", "sigma":"Conductivity (σ)", "melt":"Melting Point",
        "k":"Dielectric Constant (κ)", "Eg":"Band Gap (Eg)", "Ebd":"Breakdown Field (Ebd)",
        "bandgap":"Band Gap (Eg)", "mobility":"Electron Mobility (μ)", "thermal":"Thermal Conductivity (k)",
        "breakdown":"Breakdown Field (Ec)", "alpha":"Resistivity Temp. Coeff. (α)",
        "kth":"Thermal Conductivity (k_th)", "dEc":"Conduction-band offset to Si (ΔEc)", "dEv":"Valence-band offset to Si (ΔEv)"
    }[prop]
    if target:
        lo, hi = target
        return f"{label}<div class='sub'>[target {lo:.2g}–{hi:.2g} {unit}]</div>"
    return label

def reasons_list(props, target_ranges):
    label = {
        "phi":"Work Function (ΦM)", "sigma":"Conductivity (σ)", "melt":"Melting Point",
        "k":"Dielectric Constant (κ)", "Eg":"Band Gap (Eg)", "Ebd":"Breakdown Field (Ebd)",
        "bandgap":"Band Gap (Eg)", "mobility":"Electron Mobility (μ)", "thermal":"Thermal Conductivity (k)",
        "breakdown":"Breakdown Field (Ec)", "alpha":"Resistivity Temp. Coeff. (α)",
        "kth":"Thermal Conductivity (k_th)", "dEc":"Conduction-band offset to Si (ΔEc)", "dEv":"Valence-band offset to Si (ΔEv)"
    }
    unit = {"phi":"eV","sigma":"S/m","melt":"°C","k":"","Eg":"eV","Ebd":"MV/cm",
            "bandgap":"eV","mobility":"cm²/V·s","thermal":"W/m·K","breakdown":"MV/cm",
            "alpha":"1/°C","kth":"W/m·K","dEc":"eV","dEv":"eV"}
    items=[]
    for p in props:
        tr = target_ranges.get(p)
        tspan = f"[target {tr[0]:.2g}–{tr[1]:.2g} {unit[p]}]" if tr else ""
        items.append(f"<li><b>{label[p]}:</b> {esc(REASON[p])}. <span class='target'>{tspan}</span></li>")
    return "<ul>"+ "".join(items) +"</ul>"

def rec_list(layer, topk):
    names = [esc(item[1]["name"]) for item in topk]
    return f"Recommended {layer} materials (top 5): " + ", ".join(names) + "."

# ============================
# Main app logic
# ============================
def app_logic(goal, scenarios,
              power_pr, speed_pr, noise_pr, rel_pr, cmos_req,
              freq, freq_pr, vdd, vdd_pr, tmax, tmax_pr):

    if not scenarios:
        return "<div class='card warn'><b>Please select at least one scenario.</b></div>"

    T_low, T_high = default_T_range(scenarios, tmax)

    keys_m = pick_3_keys("metal", scenarios)
    keys_d = pick_3_keys("dielectric", scenarios)
    keys_s = pick_3_keys("semi", scenarios)

    shrink = compute_shrink_factor(scenarios, power_pr, speed_pr, noise_pr, rel_pr, freq, freq_pr, vdd, vdd_pr, tmax, tmax_pr)

    TMetal_all = merge_intersect_and_narrow(scenarios, TARGETS_METAL, ["phi","sigma","melt","alpha","kth"], shrink)
    TDiel_all  = merge_intersect_and_narrow(scenarios, TARGETS_DIEL,  ["k","Ebd","Eg","kth","dEc","dEv"], shrink)
    TSemi_all  = merge_intersect_and_narrow(scenarios, TARGETS_SEMI,  ["bandgap","mobility","thermal","breakdown"], shrink)

    # design-knob shifts
    TMetal_all = shift_targets(TMetal_all, "metal", freq, vdd, tmax)
    TDiel_all  = shift_targets(TDiel_all,  "dielectric", freq, vdd, tmax)
    TSemi_all  = shift_targets(TSemi_all,  "semi", freq, vdd, tmax)

    # keep only selected 3 keys
    TMetal = {k:TMetal_all.get(k) for k in keys_m}
    TDiel  = {k:TDiel_all.get(k)  for k in keys_d}
    TSemi  = {k:TSemi_all.get(k)  for k in keys_s}

    # candidates
    metals = METALS[:]
    dielectrics = DIELECTRICS[:]
    semis = [s for s in SEMICONDUCTORS if (not cmos_req) or (s["name"] in ("Si","Ge","SiGe"))]
    if cmos_req and not semis:
        return "<div class='card warn'><b>CMOS-only selected but no semiconductor candidates remain.</b></div>"

    # environment-aware ranges
    metal_ranges = {m["name"]: metal_range(m, T_low, T_high) for m in metals}
    diel_ranges  = {d["name"]: diel_range(d, T_low, T_high) for d in dielectrics}
    semi_ranges  = {s["name"]: semi_range(s, T_low, T_high, vdd) for s in semis}

    # elastic target boxes (ensure ≥1 in-box)
    TMetal, winner_m, r_m = ensure_min_one_in_box("metal", metals, metal_ranges, TMetal, keys_m)
    TDiel,  winner_d, r_d = ensure_min_one_in_box("dielectric", dielectrics, diel_ranges, TDiel, keys_d)
    TSemi,  winner_s, r_s = ensure_min_one_in_box("semi", semis, semi_ranges, TSemi, keys_s)

    # weights
    w_m = build_priority_weights("metal",     keys_m, power_pr, speed_pr, noise_pr, rel_pr, freq, freq_pr, vdd, vdd_pr, tmax, tmax_pr, scenarios)
    w_d = build_priority_weights("dielectric",keys_d, power_pr, speed_pr, noise_pr, rel_pr, freq, freq_pr, vdd, vdd_pr, tmax, tmax_pr, scenarios)
    w_s = build_priority_weights("semi",      keys_s, power_pr, speed_pr, noise_pr, rel_pr, freq, freq_pr, vdd, vdd_pr, tmax, tmax_pr, scenarios)

    # rankings
    top5_metal, rec5_metal = rank_layer(metals, "metal",      metal_ranges, TMetal, keys_m, topk=5, weights=w_m)
    top5_diel,  rec5_diel  = rank_layer(dielectrics, "dielectric", diel_ranges,  TDiel,  keys_d, topk=5, weights=w_d)
    top5_semi,  rec5_semi  = rank_layer(semis,  "semi",       semi_ranges,  TSemi,  keys_s, topk=5, weights=w_s)

    def build_table(layer, topk, ranges_map, targets, key_props):
        headers = ["Material"] + [header_with_target(p, targets.get(p), layer) for p in key_props]
        th = "".join([f"<th>{h}</th>" for h in headers])
        rows=[]
        for score, c, _ in topk:
            name = c["name"]; r = ranges_map[name]
            tds=[f"<td>{esc(name)}</td>"]
            for p in key_props:
                lo,hi = r[p]
                tds.append(f"<td>{fmt_range(p, lo, hi, layer)}</td>")
            rows.append(f"<tr>{''.join(tds)}</tr>")
        return f"<table class='prop-table'><thead><tr>{th}</tr></thead><tbody>{''.join(rows)}</tbody></table>"

    # reasons
    metal_reasons = reasons_list(keys_m, TMetal)
    diel_reasons  = reasons_list(keys_d, TDiel)
    semi_reasons  = reasons_list(keys_s, TSemi)

    relax_note = f" Elastic target box used — Metal:{int(100*r_m)}%, Dielectric:{int(100*r_d)}%, Semi:{int(100*r_s)}%." if (r_m or r_d or r_s) else ""

    out_html = (
        f"<div class='card'><h3>Input Summary</h3>"
        f"<p><b>Scenario(s):</b> {', '.join([esc(s) for s in scenarios])}</p>"
        f"<p><b>Design goal:</b> {esc(goal) if goal else '—'}</p>"
        f"<p><b>Requirements & advanced:</b> Power:{esc(power_pr)}; Speed:{esc(speed_pr)}; Noise:{esc(noise_pr)}; Reliability:{esc(rel_pr)}; "
        f"{'CMOS-only' if cmos_req else 'CMOS-optional'}; "
        f"{('f≈'+esc(freq)+' GHz, ') if freq else ''}{('V<sub>DD</sub>≈'+esc(vdd)+' V, ') if vdd else ''}T≈{int(T_low)}–{int(T_high)} °C.</p>"
        f"<p class='note'>Targets are built from scenario intersection → adaptive narrowing → knob-guided shifts. "
        f"Material properties are mapped to environment-aware ranges with small deterministic per-material offsets. "
        f"Scoring blends median proximity and interval overlap with mild diversity; "
        f"<b>hard filters</b> drop clearly impractical candidates. {esc(relax_note)}</p>"
        f"</div>"

        f"<div class='card'><h3>Gate Metal — Key Properties (auto-selected 3)</h3>{metal_reasons}"
        f"<h4>Top 5 Candidate Gate Metals</h4>{build_table('metal', top5_metal, metal_ranges, TMetal, keys_m)}"
        f"<p>{rec_list('gate metal', rec5_metal)}</p></div>"

        f"<div class='card'><h3>Gate Dielectric — Key Properties (auto-selected 3)</h3>{diel_reasons}"
        f"<h4>Top 5 Candidate Dielectrics</h4>{build_table('dielectric', top5_diel, diel_ranges, TDiel, keys_d)}"
        f"<p>{rec_list('dielectric', rec5_diel)}</p></div>"

        f"<div class='card'><h3>Semiconductor — Key Properties (auto-selected 3)</h3>{semi_reasons}"
        f"<h4>Top 5 Candidate Semiconductors</h4>{build_table('semi', top5_semi, semi_ranges, TSemi, keys_s)}"
        f"<p>{rec_list('semiconductor', rec5_semi)}</p></div>"
    )
    return out_html

# ============================
# UI
# ============================
CSS = """
label, .wrap { white-space: normal !important; }
.card { background:#fafafa; border:1px solid #ddd; border-radius:10px; padding:14px 16px; margin:14px 0; }
.card h3 { margin:4px 0 10px; }
.card h4 { margin:8px 0 8px; }
.card.warn { background:#fff8e6; border-color:#ffd37a; }
.prop-table { width:100%; border-collapse:collapse; }
.prop-table th, .prop-table td { border:1px solid #e3e3e3; padding:8px 10px; text-align:center; font-size:14px; vertical-align:middle; }
.prop-table th { background:#f2f2f2; }
.prop-table th .sub { font-size:12px; color:#666; margin-top:2px; }
.target { color:#555; font-size:12px; }
.note { font-size:12px; color:#666; margin-top:8px; }
.gradio-container { max-width:1040px !important; }
"""

with gr.Blocks(css=CSS) as demo:
    gr.Markdown("## MOSFET Stack-Level Material Selector (Metal–Insulator–Semiconductor) — Each layer auto-selects 3 keys from an expanded property pool")

    goal = gr.Textbox(label="Describe your target MOSFET in one sentence",
                      placeholder='e.g., "Low-power logic CMOS at 1 V, 2 GHz, high reliability"', lines=1)

    gr.Markdown(SCENARIO_GUIDE_MD)

    scenarios = gr.CheckboxGroup(label="Select application scenarios (≥1)",
                                 choices=SCENARIOS, value=[])

    with gr.Accordion("Design requirements (optional)", open=True):
        with gr.Row():
            power_pr  = gr.Radio(["Low","Medium","High"], value="Low", label="Power priority")
            speed_pr  = gr.Radio(["Low","Medium","High"], value="High", label="Speed priority")
            noise_pr  = gr.Radio(["Low","Medium","High"], value="Medium", label="Noise priority")
            rel_pr    = gr.Radio(["Low","Medium","High"], value="High", label="Reliability priority")
            cmos_req  = gr.Checkbox(label="CMOS-compatible only (Si/Ge/SiGe)", value=False)

    with gr.Accordion("Advanced parameters (optional)", open=False):
        with gr.Row():
            freq    = gr.Number(label="Target f_max (GHz)", value=None, precision=0)
            freq_pr = gr.Radio(["Low","Medium","High"], value="Medium", label="Priority")
        with gr.Row():
            vdd     = gr.Number(label="VDD (V)", value=None, precision=0)
            vdd_pr  = gr.Radio(["Low","Medium","High"], value="Medium", label="Priority")
        with gr.Row():
            tmax    = gr.Number(label="Max ambient T_max (°C)", value=None, precision=0)
            tmax_pr = gr.Radio(["Low","Medium","High"], value="Medium", label="Priority")

    run_btn = gr.Button("Generate", variant="primary")
    out = gr.HTML()

    run_btn.click(
        fn=app_logic,
        inputs=[goal, scenarios,
                power_pr, speed_pr, noise_pr, rel_pr, cmos_req,
                freq, freq_pr, vdd, vdd_pr, tmax, tmax_pr],
        outputs=out
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a818d1de8df1c94b7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
